#### Pipeline building

In [ ]:
from astropy.io import ascii
from MyToolkit import *
from Clustering_module import *
from astropy import units as u
from astropy.io import fits
import matplotlib as mpl

DP2_DIRECTORY = "/home/vibin/MyFolder/WorkDesk/DP2/"

# def make_rand_from_dist(red_col, n_samples):            #Make a random z distribution from the data z distribution
#     bins = [2.90 + i*0.005 for i in range(500)]         #using inverse transform sampling
#     hist, edges = np.histogram(red_col, bins=bins)
#     bin_widths = np.diff(edges)
#     cdf = np.cumsum(hist * bin_widths) / np.sum(hist * bin_widths)
#     #plt.stairs(cdf, bins)

#     # Generate new z values that follow the histogram distribution
#     uniform_values = np.random.rand(n_samples)
#     bin_indices = np.searchsorted(cdf, uniform_values)
#     bin_edges = edges[bin_indices-1]
#     bin_diff = edges[bin_indices] - edges[bin_indices-1]
#     bin_weights = (uniform_values - cdf[bin_indices-1]) / (cdf[bin_indices] - cdf[bin_indices-1])

#     new_z_values = bin_edges + bin_weights * bin_diff

#     return new_z_values

def make_shen_random_catalog_v3(ra_col, dec_col, red_col, init_level, mult):
    ### First made a random distribution of points in the whole sky, then chose
    ### only those points that fall in the SDSS map 
    num_quasars = len(ra_col) * mult

    nside2 = 2 ** (init_level+1)
    nside3 = 2 ** (init_level+3)
    nside4 = 2 ** (init_level+4)
    nside5 = 2 ** (init_level+6)

    npixels = hp.nside2npix(nside2)
    hp_map = np.zeros(npixels)

    occ_pix_high = hp.ang2pix(nside2, ra_col, dec_col, lonlat=True)
    hp_map[occ_pix_high] = 1
    hp_map = hp.ud_grade(hp_map, nside3)
    occ_pix_high = np.where(hp_map == 1)

    occ_pix_high = np.unique(hp.get_all_neighbours(nside3, occ_pix_high))
    occ_pix_high = np.delete(occ_pix_high, np.where(occ_pix_high == -1))
    hp_map[occ_pix_high] = 1
    hp_map = hp.ud_grade(hp_map, nside4)
    occ_pix_high = np.where(hp_map == 1)

    occ_pix_high = np.unique(hp.get_all_neighbours(nside4, occ_pix_high))
    occ_pix_high = np.delete(occ_pix_high, np.where(occ_pix_high == -1))
    hp_map[occ_pix_high] = 1
    hp_map = hp.ud_grade(hp_map, nside5)
    occ_pix_high = np.where(hp_map == 1)

    occ_pix_high = np.unique(hp.get_all_neighbours(nside5, occ_pix_high))
    occ_pix_high = np.delete(occ_pix_high, np.where(occ_pix_high == -1))
    hp_map[occ_pix_high] = 1

    # map_now = healpix_sky_map_ps1_cover_area(ra_col, dec_col, init_level)
    # map_now = map_now[1]
    map_now = hp_map
    nside = hp.get_nside(map_now)

    pixels_with_data = np.shape(np.where(map_now == 1))[1]
    area_data_pixels = pixels_with_data * hp.nside2pixarea(hp.get_nside(map_now), degrees=True)
    print("Mask area = ", area_data_pixels)

    ind = np.where(np.arange(num_quasars) > -1)
    rand_ra = np.zeros(num_quasars)
    rand_dec = np.zeros(num_quasars)

    while True:
        rand_ra[ind] = np.random.uniform(0, 360, len(ind[0]))
        rand_dec[ind] = np.degrees(np.arcsin(np.random.uniform(-0.5, 1, len(ind[0]))))     

        pix_of_points = hp.ang2pix(nside, np.absolute(np.radians(rand_dec) - np.pi/2), np.radians(rand_ra))
        ind = np.where(map_now[pix_of_points] == 0)

        if(len(ind[0]) == 0):
            break 

    rand_red = make_rand_from_dist(red_col, num_quasars)

    return rand_ra, rand_dec, rand_red

def make_shen_random_catalog_v2(ra_col, dec_col, red_col, init_level, mult):
    ### First made a random distribution of points in the whole sky, then chose
    ### only those points that fall in the SDSS map 
    num_quasars = len(ra_col) * mult

    nside = 2 ** init_level

    map_now = healpix_sky_map_ps1_cover_area(ra_col, dec_col, init_level)
    map_now = map_now[1]

    ind = np.where(np.arange(num_quasars) > -1)
    rand_ra = np.zeros(num_quasars)
    rand_dec = np.zeros(num_quasars)

    while True:
        rand_ra[ind] = np.random.uniform(0, 360, len(ind[0]))
        rand_dec[ind] = np.degrees(np.arcsin(np.random.uniform(-0.5, 1, len(ind[0]))))     

        pix_of_points = hp.ang2pix(nside, np.absolute(np.radians(rand_dec) - np.pi/2), np.radians(rand_ra))
        ind = np.where(map_now[pix_of_points] == 0)

        if(len(ind[0]) == 0):
            break 

    rand_red = make_rand_from_dist(red_col, num_quasars)

    return rand_ra, rand_dec, rand_red

def make_shen_random_catalog(ra_col, dec_col, red_col, init_level, resol_upgrd, mult):
    ### Healpy pixelated the data catalog to find the rough footprint, appropriate healpy level found
    ### from the graph of coverage area vs level. Then increased the pixelation and chosen random
    ### pixels to assign points to its centre.
    ### 
    num_quasars = len(ra_col) * mult

    map_now = healpix_sky_map_ps1_cover_area(ra_col, dec_col, init_level)
    map_now = map_now[1]

    map_then = hp.pixelfunc.ud_grade(map_now, 2 ** resol_upgrd)

    map_random = np.zeros_like(map_then)
    val_pos = np.where(map_then != 0)
    num_pix = len(val_pos[0])

    rand = np.random.randint(0, num_pix, num_quasars)
    rand_pos = val_pos[0][rand]
    map_random[rand_pos] = 1

    rand_ang = hp.pixelfunc.pix2ang(2**resol_upgrd, rand_pos, lonlat=True)

    rand_ra_col = rand_ang[0]
    rand_dec_col = rand_ang[1]

    rand_red_col = make_rand_from_dist(red_col, num_quasars)
    #plt.close()

    return rand_ra_col, rand_dec_col, rand_red_col

def plot_red_dist(red_col, rand_red_col, bins, draw_ax, den=False):
    red_col_shen = red_col
    rand_red_col_shen = rand_red_col

    hist, edges = np.histogram(red_col_shen, bins=bins)
    hist_good, edges_good = np.histogram(red_col_shen_good, bins=bins)

    hist_rand, edges_rand = np.histogram(rand_red_col_shen, bins=bins)

    #print(edges)
    if (den==False):
        hist = hist/np.max(hist)
        hist_good = hist_good/np.max(hist_good)
        hist_rand = hist_rand/np.max(hist_rand)

        ax_fin_cl1 = draw_ax
        ax_fin_cl1.stairs(hist, edges, label="all fields")
        #ax_fin_cl1.stairs(hist_good, edges, label="good field only")
        ax_fin_cl1.stairs(hist_rand, edges, label="random cat")
        ax_fin_cl1.set_xlabel('z')
        ax_fin_cl1.set_ylabel('Distribution')
        ax_fin_cl1.legend()
    else :
        ax_fin_cl1 = draw_ax
        ax_fin_cl1.hist(red_col_shen, bins=bins, histtype='step', density=True, label="all fields")
        #ax_fin_cl1.stairs(hist_good, edges, label="good field only")
        ax_fin_cl1.hist(rand_red_col_shen, bins=bins, histtype='step', density=True, label="random cat")
        ax_fin_cl1.set_xlabel('z')
        ax_fin_cl1.set_ylabel('Distribution')
        ax_fin_cl1.legend()

def make_rand_from_dist(red_col, n_samples):            #Make a random z distribution from the data z distribution
    bins = [2.90 + i*0.005 for i in range(500)]         #using inverse transform sampling
    hist, edges = np.histogram(red_col, bins=bins)
    bin_widths = np.diff(edges)
    cdf = np.cumsum(hist * bin_widths) / np.sum(hist * bin_widths)
    #plt.stairs(cdf, bins)

    # Generate new z values that follow the histogram distribution
    uniform_values = np.random.rand(n_samples)
    bin_indices = np.searchsorted(cdf, uniform_values)
    bin_edges = edges[bin_indices-1]
    bin_diff = edges[bin_indices] - edges[bin_indices-1]
    bin_weights = (uniform_values - cdf[bin_indices-1]) / (cdf[bin_indices] - cdf[bin_indices-1])

    new_z_values = bin_edges + bin_weights * bin_diff

    ##---------------------------------------------
    ## Making changes to see how clustering changes
    #new_z_values = np.random.uniform(2.9, 3.5, n_samples)
    #new_z_values[300:] = new_z_values[300:] * 1.01
    ##

    return new_z_values


def try_clustering_for_z_dist(rand_ra_col_shen, rand_dec_col_shen, rand_red_col_shen):
    mpl.rcParams['font.size'] = 18.0
    #-------------------------------------------------------------

    bins = [2.901 + i*0.05 for i in range(50)]
    # fig_fin_cl, (ax_fin_cl1, ax_fin_cl2) = plt.subplots(1, 2, figsize=(12,5))
    fig_fin_cl, ax_fin_cl2 = plt.subplots(figsize=(7,7))
    # plot_red_dist(red_col_shen, rand_red_col_shen, bins, ax_fin_cl1, den=True)

    #-------------------------------------------------------------

    s_bins = np.logspace(start=np.log10(1.9868), stop=np.log10(314.915), num=23)
    s_mid = (s_bins[:-1] + s_bins[1:])/2

    original = ma.masked_values([0, 0, 0, 16.5, 0, 3.54, 1.26, 0.663, 0.191, 0.131, 0.236, -0.280, 0.361, 0.101, 0.0384, 0.0368, 0.0101, 0.0194, -0.00396, 0.0101, -0.00296, 0.00214], 0)
    original_error = ma.masked_values([0, 0, 0, 12.8, 0, 3.61, 1.88, 0.733, 0.786, 0.472, 0.175, 0.223, 0.170, 0.121, 0.0862, 0.0644, 0.0382, \
                                       0.0250, 0.0219, 0.0134, 0.00672, 0.00953], 0)

    fit_clust = pow(s_mid/ 10.2, -1.71)

    ax_fin_cl2.plot(s_mid, fit_clust, '--', label="Shen fit")
    ax_fin_cl2.errorbar(s_mid*np.power(10, 0.02), original, original_error, fmt='r*', capsize=5, label="Shen data")

    #----------------------------------------------------------------

    find_xi_s(ra_col_shen, dec_col_shen, red_col_shen, s_bins, "shen_redshift_space_corr_table_real_signal",\
                        rand_ra=rand_ra_col_shen, rand_dec=rand_dec_col_shen, rand_red=rand_red_col_shen, draw_ax=ax_fin_cl2)

    ax_fin_cl2.legend()

### Obtaining the data catalog from table given in Shen et al 2007
shen_file_name = DP2_DIRECTORY + "Data/shen_quasar_sample_datafile1.txt"
shen_data = ascii.read(shen_file_name)
sub_flag = shen_data.columns[8]
good_flag = shen_data.columns[9]
pos = np.where(sub_flag == 1)
pos_good = np.where(good_flag == 1)
ra_col_shen = shen_data.columns[3][pos].value
dec_col_shen = shen_data.columns[4][pos].value
red_col_shen = shen_data.columns[5][pos].value

ra_col_shen_good = shen_data.columns[3][pos_good].value
dec_col_shen_good = shen_data.columns[4][pos_good].value
red_col_shen_good = shen_data.columns[5][pos_good].value

## Trying to find clustering from Shen quasar sample

##### Obtaining the Shen quasar sample

In [ ]:
shen_file_name = DP2_DIRECTORY + "Data/shen_quasar_sample_datafile1.txt"
shen_data = ascii.read(shen_file_name)
sub_flag = shen_data.columns[8]
good_flag = shen_data.columns[9]
pos = np.where(sub_flag == 1)
pos_good = np.where(good_flag == 1)
ra_col_shen = shen_data.columns[3][pos].value
dec_col_shen = shen_data.columns[4][pos].value
red_col_shen = shen_data.columns[5][pos].value

ra_col_shen_good = shen_data.columns[3][pos_good].value
dec_col_shen_good = shen_data.columns[4][pos_good].value
red_col_shen_good = shen_data.columns[5][pos_good].value

print("There are {} quasars in the catalog".format(len(ra_col_shen)))
ax = plot_sky_map_ps1(ra_col_shen, dec_col_shen, labels=['quasars'], title="Shen High redshift quasar catalog", mark_size=0.1)

In [ ]:
bins = [2.9 + i*0.05 for i in range(50)]
hist, edges = np.histogram(red_col_shen, bins=bins)
hist_good, edges_good = np.histogram(red_col_shen_good, bins=bins)
print(edges)
hist = hist/np.max(hist)
hist_good = hist_good/np.max(hist_good)

plt.stairs(hist, edges, label="all fields")
plt.stairs(hist_good, edges, label="good field only")
plt.title("Redshift distribution of Shen quasars")
plt.legend()

In [ ]:
red_col_shen.max()

##### Finding the footprint

In [ ]:
level = []
cov_area = []
print("Level    coverage area")
for i in [3,4,5,6,7,8,9,10]:
    map_i, wat = healpix_sky_map_ps1_cover_area(ra_col_shen, dec_col_shen, i, title="Level " + str(i))
    level.append(i)
    cov_area.append(map_i)
    print(f'{i:5d}      {map_i:8.5f}')

print(cov_area)
fig, ax = plt.subplots(figsize = (12, 7))
ax.plot(level, cov_area, '*-')
ax.set_title("Effective coverage area versus healpix level for Shen complete quasar sample")
ax.set_xlabel("Level")
ax.set_ylabel("Effective coverage area")

##### Upgrading the footprint finding method

In [ ]:
level = 5
nside1 = 2**level
nside2 = 2**(level + 1)
nside3 = 2**(level + 3)
nside4 = 2**(level + 4)
nside5 = 2**(level + 6)

npixels = hp.nside2npix(nside1)
hp_map = np.zeros(npixels)

ra = ra_col_shen
dec = dec_col_shen

occ_pix = hp.ang2pix(nside1, ra, dec, lonlat=True)
hp_map[occ_pix] = 1
hp_map = hp.ud_grade(hp_map, nside2)

occ_pix_high = hp.ang2pix(nside2, ra, dec, lonlat=True)
hp_map[occ_pix_high] = 0.5
hp_map = hp.ud_grade(hp_map, nside3)
occ_pix_high = np.where(hp_map == 0.5)


occ_pix_high_bound = np.unique(hp.get_all_neighbours(nside3, occ_pix_high))
occ_pix_high_bound = np.delete(occ_pix_high_bound, np.where(occ_pix_high_bound == -1))
hp_map[occ_pix_high_bound] = 0.5
hp_map = hp.ud_grade(hp_map, nside4)
occ_pix_high_bound = np.where(hp_map == 0.5)

occ_pix_high_bound = np.unique(hp.get_all_neighbours(nside4, occ_pix_high_bound))
occ_pix_high_bound = np.delete(occ_pix_high_bound, np.where(occ_pix_high_bound == -1))
hp_map[occ_pix_high_bound] = 0.5
hp_map = hp.ud_grade(hp_map, nside5)
occ_pix_high_bound = np.where(hp_map == 0.5)

occ_pix_high_bound = np.unique(hp.get_all_neighbours(nside5, occ_pix_high_bound))
occ_pix_high_bound = np.delete(occ_pix_high_bound, np.where(occ_pix_high_bound == -1))
hp_map[occ_pix_high_bound] = 0.5

# temp_ra, temp_dec = hp.pix2ang(nside3, occ_pix_high_bound, lonlat=True)
# print(np.where(occ_pix_high_bound == -1))
# occ_pix_high_bound_extra = np.unique(hp.get_all_neighbours(nside3, temp_theta[1:], temp_phi[1:]))

# hp_map = hp.ud_grade(hp_map, nside3)


# hp_map[occ_pix_high_bound_extra] = 0.5

pixels_with_data = np.shape(np.where(hp_map == 0.5))[1]
area_data_pixels = pixels_with_data * hp.nside2pixarea(hp.get_nside(hp_map), degrees=True)
print(area_data_pixels)

hp.mollview(hp_map)

In [ ]:
hp_map_up = np.zeros(hp.nside2npix(hp.get_nside(hp_map)))
hp_map_up[np.where(hp_map == 0.5)] = 1

# nside_big = 2 ** 14
# hp_map_up = hp.ud_grade(hp_map_up, nside_big)

hp.mollview(hp_map_up, title="Best approximation of PS1 footprint")

##### Clustering measurement for the upgraded footprint----- BEST ONE YET

In [ ]:
rand_ra_col_shen, rand_dec_col_shen, rand_red_col_shen = make_shen_random_catalog_v3(ra_col_shen, dec_col_shen, red_col_shen, 5, 20)

try_clustering_for_z_dist(rand_ra_col_shen, rand_dec_col_shen, rand_red_col_shen)

In [ ]:
rand_ra_col_shen, rand_dec_col_shen, rand_red_col_shen = make_shen_random_catalog_v3(ra_col_shen, dec_col_shen, red_col_shen, 5, 1)
plot_sky_map_ps1(rand_ra_col_shen, rand_dec_col_shen, ['quasars'], title="Random catalog made for shen quasar sample", mark_size=0.1)

In [ ]:
step = 0.05
bins = [2.901 + i*0.05 for i in range(50)]

with plt.style.context('bmh'):
    fig, ax = plt.subplots(figsize=(7,7))
    # hist, bins = np.histogram(red_col_schind_23, bins)
    # hist = hist/np.max(hist)
    ax.hist(red_col_shen, bins, density=True, histtype='step', zorder=10, alpha=1, label = 'Shen sample')
    ax.grid(zorder=1, alpha=0.3)
    ax.set_xlabel('redshift')
    ax.set_ylabel('number density')
    ax.set_aspect('auto')
    plt.rcParams.update({'font.size': 12})

    ax.hist(rand_red_col_shen, bins, density=True, histtype='step', zorder=10, alpha =1, label = 'random sample')
    ax.grid(zorder=1, alpha=0.3)
    ax.set_xlabel('redshift')
    ax.set_ylabel('normalized number count')
    ax.set_aspect('auto')

    plt.legend()

##### Plot: distribution of random catalog

In [ ]:
def plot_sky_map_ps1_v2(ra, dec, mask = None, title='', labels=['quasars'], mark_size=30):

    if (mask == None):
        mask = np.zeros_like(ra, dtype='int')
        mask = np.insert(mask, 0, 1)

    ### Plots the lines marking the excluded area of the galactic plane
    res = 511
    gal_l = np.linspace(0, 360, res)
    gal_b = np.ones(res) * 20.0
    gal_line_1 = SkyCoord(gal_l, gal_b, frame = 'galactic', unit = 'deg')
    gal_line_2 = SkyCoord(gal_l, -gal_b, frame = 'galactic', unit = 'deg')
    ### gal_line_equat is the galactic line in equatorial coordinates
    gal_line_equat_1 = gal_line_1.transform_to('icrs')   
    gal_line_equat_2 = gal_line_2.transform_to('icrs')
    gal_ra_1 = np.radians(range_wrapper(gal_line_equat_1.ra.degree))
    ###Sorting the array in order to do line plot
    i_sorted = np.argsort(gal_ra_1) 
    gal_ra_1 = gal_ra_1[i_sorted]
    gal_dec_1 = np.radians(gal_line_equat_1.dec.degree)[i_sorted]
    gal_ra_2 = np.radians(range_wrapper(gal_line_equat_2.ra.degree))
    i_sorted = np.argsort(gal_ra_2)
    gal_ra_2 = gal_ra_2[i_sorted]
    gal_dec_2 = np.radians(gal_line_equat_2.dec.degree)[i_sorted]
    ### dec -30 array is obtained
    dec_30_ra = np.radians(range_wrapper(np.linspace(0, 360, res)))
    dec_30_dec = np.radians(np.ones(res) * -30.0)

    fig, ax = plt.subplots(figsize = (12, 7), subplot_kw={'projection': 'mollweide'})
    plt.rcParams.update({'font.size': 12})
    plt.rc('legend', fontsize=14)
    #ax = plt.subplot(111, projection = "mollweide")

    ### This plots the footprint
    ax.plot(gal_ra_1, gal_dec_1, color = 'black', linestyle='--', linewidth = 1) 
    ax.plot(gal_ra_2, gal_dec_2, color = 'black', linestyle='--', linewidth = 1)
    # ax.plot(dec_30_ra, dec_30_dec, color = 'black', linestyle='-.', linewidth = 1)
    #-------------------------------------------------------------------------------------------
    ### Plotting the sources
    ra = np.radians(range_wrapper(ra)) 
    dec = np.radians(dec)
    marker = ['o','x']

    for n_source in range(mask[0]):
        source_pos = np.where(mask[1:] == n_source)
        ra_n, dec_n = ra[source_pos], dec[source_pos]

        ax.scatter(ra_n, dec_n, marker = marker[n_source], label = labels[n_source], color='navy', alpha=1, s = mark_size, zorder=10)

    # foot_ra, foot_dec, foot_red = make_shen_random_catalog_v3(ra_col_shen, dec_col_shen, red_col_shen, 5, 50)
    # foot_ra=np.radians(range_wrapper(foot_ra))
    # foot_dec = np.radians(foot_dec)
    # ax.scatter(foot_ra, foot_dec, color='grey', s=0.1, zorder=2, alpha = 0.05)  

    ax.set_title(title)
    ax.legend()
    ax.grid(True)

    ax.set_xticklabels(["10h", "8h", "6h", "4h", "2h", "0h", "22h", "20h", "18h", "16h", "14h"]);
    #plt.style.use(astropy_mpl_style)

    return ax

ax = plot_sky_map_ps1_v2(ra_col_shen, dec_col_shen,labels = ['quasars'], title="", mark_size=0.5)




##### Showing our DD pair counts agrees with that of Shen

In [ ]:
s_bins = np.logspace(start=np.log10(1.9868), stop=np.log10(314.915), num=23)
s_mid = (s_bins[:-1] + s_bins[1:])/2

s_array = find_s_bined(ra_col_shen, dec_col_shen, red_col_shen)

s_hist, edges = np.histogram(s_array, s_bins)

In [ ]:
shen_DD_original = np.array([0.0, 0.0, 0.0, 1.8, 0.0, 1.8, 1.8, 2.7, 4.5, 8.9, 15.2, 22.4, 70.7, 104.9, 210.9, 384.8, 734.2, 1417.1, 2565.8, 4821.6, 8631.8, 15365.1])
shen_RR_original = np.array([0.9, 5.4, 6.3, 14.4, 34.2, 38.7, 99.0, 215.0, 406.5, 804.2, 1592.4,\
                              3123.6, 6028.6, 11959.1, 23480.2, 45648.7, 88337.9, 168480.9, 3117727.8, 588892.8, 1070807.1, 1912774.1])
plt.stairs(s_hist, s_bins)
plt.stairs(shen_DD_original, s_bins)
plt.xlabel('s')
plt.ylabel('DD')

print((s_hist - shen_DD_original)/s_hist)

### Trying to make a random catalog for Shen quasars

#### random catalog examples

In [ ]:
rand_ra_col_shen, rand_dec_col_shen, rand_red_col_shen = make_shen_random_catalog(ra_col_shen, dec_col_shen, red_col_shen, 5, 11, 2)
plt.close()

plot_sky_map_ps1(rand_ra_col_shen, rand_dec_col_shen, labels=['quasars'], title="Random catalog made for Shen quasar sample", mark_size=0.1)

In [ ]:
### Random catalog making v2

rand_ra_col_shen, rand_dec_col_shen, rand_red_col_shen = make_shen_random_catalog_v2(ra_col_shen, dec_col_shen, red_col_shen, 5, 1)
plt.close()

plot_sky_map_ps1(rand_ra_col_shen, rand_dec_col_shen, labels=['quasars'], title="Random catalog made for Shen quasar sample", mark_size=0.1)

#### Redshift distribution of the random catalog from inverse tranform sampling

In [ ]:
rand_ra_col_shen, rand_dec_col_shen, rand_red_col_shen = make_shen_random_catalog(ra_col_shen, dec_col_shen, red_col_shen, 5, 11, 2)
plt.close()

bins = [2.901 + i*0.05 for i in range(50)]
hist, edges = np.histogram(red_col_shen, bins=bins)
hist_good, edges_good = np.histogram(red_col_shen_good, bins=bins)

hist_rand, edges_rand = np.histogram(rand_red_col_shen, bins=bins)

#print(edges)
hist = hist/np.max(hist)
hist_good = hist_good/np.max(hist_good)
hist_rand = hist_rand/np.max(hist_rand)

plt.stairs(hist, edges, label="all fields")
#plt.stairs(hist_good, edges, label="good field only")
plt.stairs(hist_rand, edges, label="random cat")
plt.xlabel('z')
plt.ylabel('Distribution')
plt.legend()

#### Analysing the random catalog made

In [ ]:
### converting the random catalog to a bunch of x, y and z in comoving coordinates

rand_ra_col, rand_dec_col, rand_red_col = make_shen_random_catalog_v2(ra_col_shen, dec_col_shen, red_col_shen, 5, 12)
plt.close()

s_bins = np.logspace(start=np.log10(1.9868), stop=np.log10(314.915), num=23)

s_hist = (pair_count_corrfunc(ra_col_shen, dec_col_shen, red_col_shen, s_bins)[:,3]).astype(int)
rand_s_hist = (pair_count_corrfunc(rand_ra_col, rand_dec_col, rand_red_col, s_bins)[:,3]).astype(int)
cross_s_hist = (pair_count_corrfunc(ra_col_shen, dec_col_shen, red_col_shen, s_bins, rand_ra_col=rand_ra_col, rand_dec_col=rand_dec_col, rand_red_col=rand_red_col)[:,3]).astype(int) * 2
s_hist_norm = (len(ra_col_shen) * (len(ra_col_shen) - 1))/2
rand_hist_norm = (len(rand_ra_col) * (len(rand_ra_col) - 1))/2
cross_hist_norm = len(ra_col_shen) * len(rand_ra_col)

unfin_pos = np.where(s_hist * rand_s_hist * cross_s_hist == 0)

with np.errstate(divide='ignore', invalid='ignore'):
    xi_s = (s_hist/s_hist_norm - 2 * cross_s_hist/cross_hist_norm + rand_s_hist/rand_hist_norm)/(rand_s_hist/rand_hist_norm)
xi_s_masked = ma.array(xi_s)
xi_s_masked[unfin_pos] = -999

printer = np.array([s_hist, rand_s_hist, cross_s_hist, xi_s_masked]).T
print(printer)
print(rand_hist_norm/s_hist_norm)

# binned_rand = find_s_bined(rand_ra_col, rand_dec_col, rand_red_col)
# hist_rand = np.histogram(binned_rand, s_bins)
# print(hist_rand[0])

##### ----------------------------------------------------------------------------------------------------

#### Best attempt on cluster measurement so far

In [ ]:
# rand_ra_col_shen, rand_dec_col_shen, rand_red_col_shen = make_shen_random_catalog(ra_col_shen, dec_col_shen, red_col_shen, 5, 11, 1)
# plt.close()

def make_rand_from_dist(red_col, n_samples):            #Make a random z distribution from the data z distribution
    bins = [2.90 + i*0.005 for i in range(500)]         #using inverse transform sampling
    hist, edges = np.histogram(red_col, bins=bins)
    print(np.shape(hist))
    bin_widths = np.diff(edges)
    cdf = np.cumsum(hist * bin_widths) / np.sum(hist * bin_widths)
    cdf = np.insert(cdf, 0, 0)
    #plt.stairs(cdf, bins)

    # Generate new z values that follow the histogram distribution
    uniform_values = np.random.rand(n_samples)
    bin_indices = np.searchsorted(cdf, uniform_values)
    bin_edges = edges[bin_indices-1]
    bin_diff = edges[bin_indices] - edges[bin_indices-1]
    bin_weights = (uniform_values - cdf[bin_indices-1]) / (cdf[bin_indices] - cdf[bin_indices-1])

    new_z_values = bin_edges + bin_weights * bin_diff

    ##---------------------------------------------
    ## Making changes to see how clustering changes
    #new_z_values = np.random.uniform(2.9, 3.5, n_samples)
    #new_z_values[300:] = new_z_values[300:] * 1.01
    ##

    return new_z_values

### Trying the clustering with the v2 of random catalog maker
rand_ra_col_shen, rand_dec_col_shen, rand_red_col_shen = make_shen_random_catalog_v2(ra_col_shen, dec_col_shen, red_col_shen, 5, 10)
plt.close()

#-------------------------------------------------------------

bins = [2.901 + i*0.05 for i in range(50)]
fig_fin_cl, (ax_fin_cl1, ax_fin_cl2) = plt.subplots(1, 2, figsize=(12,5))
plot_red_dist(red_col_shen, rand_red_col_shen, bins, ax_fin_cl1)

#-------------------------------------------------------------

s_bins = np.logspace(start=np.log10(1.9868), stop=np.log10(314.915), num=23)
s_mid = (s_bins[:-1] + s_bins[1:])/2

original = ma.masked_values([0, 0, 0, 16.5, 0, 3.54, 1.26, 0.663, 0.191, 0.131, 0.236, 0, 0.361, 0.101, 0.0384, 0.0368, 0.0101, 0.0194, 0, 0.0101, 0, 0.00214], 0)

fit_clust = pow(s_mid/ 10.2, -1.71)

ax_fin_cl2.plot(s_mid, fit_clust, '--', label="Shen fit")
ax_fin_cl2.plot(s_mid, original, 'r*', label="Shen data")

#----------------------------------------------------------------

find_xi_s(ra_col_shen, dec_col_shen, red_col_shen, s_bins, "shen_redshift_space_corr_table_real_signal",\
                     rand_ra=rand_ra_col_shen, rand_dec=rand_dec_col_shen, rand_red=rand_red_col_shen, draw_ax=ax_fin_cl2)

ax_fin_cl2.legend()


#### Trying to find the probability, $P(z)$ of quasars from selection function $S(z)$ and the redshift evolution of density due to expansion of universe $\rho(z) = \rho(0)(1+z)^3$

In [ ]:
sdss_sel_fun_file_name = DP2_DIRECTORY + 'Data/sdss_q_sel_fun_richards.dat'
sdss_sel_fun_file = open(sdss_sel_fun_file_name, 'r')
sdss_sel_fun = np.genfromtxt(sdss_sel_fun_file)

sel_fun_imag_col = sdss_sel_fun[:,0]
sel_fun_z_col = sdss_sel_fun[:,1]
sel_fun_point_col = sdss_sel_fun[:,2]
sel_fun_radio_col = sdss_sel_fun[:,3]
sel_fun_extended_col = sdss_sel_fun[:,4]

### selection function is given in the following way: 53 i magnitudes are there from 15.0 to 20.2, difference of 0.1, for each magnitude 121 z values 
# from 0.0 to 6.0 is given with difference of 0.05.

sel_fun_z = sel_fun_z_col[0:121] ### this is the z values where we have the S(z)
sel_fun_imag = np.reshape(sel_fun_imag_col, (53,121))[:,0]
sel_fun_point_val = np.sum(np.reshape(sel_fun_point_col,(53, 121)), axis=0) * 0.1
sel_fun_point_grid = np.reshape(sel_fun_point_col,(53, 121))
sel_fun_radio_val = np.sum(np.reshape(sel_fun_radio_col,(53, 121)), axis=0) * 0.1
sel_fun_ext_val = np.sum(np.reshape(sel_fun_extended_col,(53, 121)), axis=0) * 0.1

loc_29_54 = np.where((sel_fun_z >= 2.9) & (sel_fun_z <= 5.4))
sel_fun_z_act= sel_fun_z[loc_29_54][1:-1]
sel_fun_z_bins = (sel_fun_z[loc_29_54] - 0.05/2)[1:]
sel_fun_point_act = (sel_fun_point_val[loc_29_54])[1:-1]

In [ ]:
numerator = np.power(1 + sel_fun_z_act, 3) * sel_fun_point_act
denominator = np.sum(numerator) * 0.1
pz = numerator/denominator
numerator.shape

fig1, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2, figsize=(10,10)) #, sharex=True, sharey=True)
ax1.plot(sel_fun_z_act, sel_fun_point_act)
ax1.set_title('selection function')
ax2.plot(sel_fun_z_act, np.power(1+sel_fun_z_act, 3))
ax2.set_title(r'$(1 + z)^3$')
ax3.plot(sel_fun_z_act, numerator)
ax3.set_title('unnormalised P(z)')
ax4.plot(sel_fun_z_act, pz)
ax4.set_title('normalised P(z)')
ax3.set_xlabel('z')

plt.show()

In [ ]:
def make_rand_from_pz(pz, bins, n_samples):
    hist, edges = pz, bins
    bin_widths = np.diff(edges)
    cdf = np.cumsum(hist * bin_widths) / np.sum(hist * bin_widths)
    #plt.stairs(cdf, bins)

    # Generate new z values that follow the histogram distribution
    uniform_values = np.random.rand(n_samples)
    bin_indices = np.searchsorted(cdf, uniform_values)
    bin_edges = edges[bin_indices-1]
    bin_diff = edges[bin_indices] - edges[bin_indices-1]
    bin_weights = (uniform_values - cdf[bin_indices-1]) / (cdf[bin_indices] - cdf[bin_indices-1])

    new_z_values = bin_edges + bin_weights * bin_diff

    return new_z_values


In [ ]:
rand_ra_col_shen, rand_dec_col_shen, rand_red_col_shen = make_shen_random_catalog_v2(ra_col_shen, dec_col_shen, red_col_shen, 5, 10)
plt.close()
rand_red_col_shen = make_rand_from_pz(pz, sel_fun_z_bins, len(rand_ra_col_shen))

#-------------------------------------------------------------

bins = [2.901 + i*0.05 for i in range(50)]
fig_fin_cl, (ax_fin_cl1, ax_fin_cl2) = plt.subplots(1, 2, figsize=(12,5))
plot_red_dist(red_col_shen, rand_red_col_shen, bins, ax_fin_cl1)

#-------------------------------------------------------------

s_bins = np.logspace(start=np.log10(1.9868), stop=np.log10(314.915), num=23)
s_mid = (s_bins[:-1] + s_bins[1:])/2

original = ma.masked_values([0, 0, 0, 16.5, 0, 3.54, 1.26, 0.663, 0.191, 0.131, 0.236, 0, 0.361, 0.101, 0.0384, 0.0368, 0.0101, 0.0194, 0, 0.0101, 0, 0.00214], 0)

fit_clust = pow(s_mid/ 10.2, -1.71)

ax_fin_cl2.plot(s_mid, fit_clust, '--', label="Shen fit")
ax_fin_cl2.plot(s_mid, original, 'r*', label="Shen data")

#----------------------------------------------------------------

find_xi_s(ra_col_shen, dec_col_shen, red_col_shen, s_bins, "shen_redshift_space_corr_table_real_signal",\
                     rand_ra=rand_ra_col_shen, rand_dec=rand_dec_col_shen, rand_red=rand_red_col_shen, draw_ax=ax_fin_cl2)

ax_fin_cl2.legend()

#### Trial and error to see how the clustering measure differs with redshift distribution

In [ ]:
def make_rand_from_dist(red_col, n_samples):            #Make a random z distribution from the data z distribution
    bins = [2.90 + i*0.0005 for i in range(5000)]         #using inverse transform sampling
    hist, edges = np.histogram(red_col, bins=bins)
    
    print(hist)
    #hist[np.where(hist < 5)] = hist[np.where(hist < 5)] - 0

    bin_widths = np.diff(edges)
    cdf = np.cumsum(hist * bin_widths) / np.sum(hist * bin_widths)
    #plt.stairs(cdf, bins)

    # Generate new z values that follow the histogram distribution
    uniform_values = np.random.rand(n_samples)
    bin_indices = np.searchsorted(cdf, uniform_values)
    bin_edges = edges[bin_indices-1]
    bin_diff = edges[bin_indices] - edges[bin_indices-1]
    bin_weights = (uniform_values - cdf[bin_indices-1]) / (cdf[bin_indices] - cdf[bin_indices-1])

    new_z_values = bin_edges + bin_weights * bin_diff

    ##---------------------------------------------
    ## Making changes to see how clustering changes
    #new_z_values = np.random.uniform(2.9, 3.5, n_samples)
    #new_z_values[300:] = new_z_values[300:] * 1.01
    ##

    return new_z_values

### Trying the clustering with the v2 of random catalog maker
rand_ra_col_shen, rand_dec_col_shen, rand_red_col_shen = make_shen_random_catalog_v2(ra_col_shen, dec_col_shen, red_col_shen, 5, 10)
plt.close()

#-------------------------------------------------------------

bins = [2.901 + i*0.05 for i in range(50)]
fig_fin_cl, (ax_fin_cl1, ax_fin_cl2) = plt.subplots(1, 2, figsize=(12,5))
# plot_red_dist(red_col_shen, rand_red_col_shen, bins, ax_fin_cl1)
ax_fin_cl1.hist(red_col_shen, bins, histtype='step', density=True)
ax_fin_cl1.hist(rand_red_col_shen, bins, histtype='step', density=True)

#-------------------------------------------------------------

s_bins = np.logspace(start=np.log10(1.9868), stop=np.log10(314.915), num=23)
s_mid = (s_bins[:-1] + s_bins[1:])/2

original = ma.masked_values([0, 0, 0, 16.5, 0, 3.54, 1.26, 0.663, 0.191, 0.131, 0.236, 0, 0.361, 0.101, 0.0384, 0.0368, 0.0101, 0.0194, 0, 0.0101, 0, 0.00214], 0)

fit_clust = pow(s_mid/ 10.2, -1.71)

ax_fin_cl2.plot(s_mid, fit_clust, '--', label="Shen fit")
ax_fin_cl2.plot(s_mid, original, 'r*', label="Shen data")

#----------------------------------------------------------------

find_xi_s(ra_col_shen, dec_col_shen, red_col_shen, s_bins, "shen_redshift_space_corr_table_real_signal",\
                     rand_ra=rand_ra_col_shen, rand_dec=rand_dec_col_shen, rand_red=rand_red_col_shen, draw_ax=ax_fin_cl2)

ax_fin_cl2.legend()


#### Dealing with the quasar selection function as given in Richards et al 2006.

In [ ]:
sdss_sel_fun_file_name = DP2_DIRECTORY + 'Data/sdss_q_sel_fun_richards.dat'
sdss_sel_fun_file = open(sdss_sel_fun_file_name, 'r')
sdss_sel_fun = np.genfromtxt(sdss_sel_fun_file)

sel_fun_imag_col = sdss_sel_fun[:,0]
sel_fun_z_col = sdss_sel_fun[:,1]
sel_fun_point_col = sdss_sel_fun[:,2]
sel_fun_radio_col = sdss_sel_fun[:,3]
sel_fun_extended_col = sdss_sel_fun[:,4]

### selection function is given in the following way: 53 i magnitudes are there from 15.0 to 20.2, difference of 0.1, for each magnitude 121 z values 
# from 0.0 to 6.0 is given with difference of 0.05.

sel_fun_z = sel_fun_z_col[0:121]
sel_fun_imag = np.reshape(sel_fun_imag_col, (53,121))[:,0]
sel_fun_point_val = np.sum(np.reshape(sel_fun_point_col,(53, 121)), axis=0) * 0.1
sel_fun_point_grid = np.reshape(sel_fun_point_col,(53, 121))
sel_fun_radio_val = np.sum(np.reshape(sel_fun_radio_col,(53, 121)), axis=0) * 0.1
sel_fun_ext_val = np.sum(np.reshape(sel_fun_extended_col,(53, 121)), axis=0) * 0.1

In [ ]:
fig1, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2, figsize=(10,10), sharex=True, sharey=True)
ax1.plot(sel_fun_z, sel_fun_point_val)
ax1.set_title('point')
ax2.plot(sel_fun_z, sel_fun_radio_val)
ax2.set_title('radio')
ax3.plot(sel_fun_z, sel_fun_ext_val)
ax3.set_title('extended')
ax4.plot(sel_fun_z, (sel_fun_point_val + sel_fun_radio_val + sel_fun_ext_val)/3)
ax4.set_title('average of all')
ax3.set_xlabel('z')
ax1.set_ylabel('fraction')
plt.show()

In [ ]:
loc_z = (sel_fun_z >= 2.9) & (sel_fun_z <= 5.4)
sel_fun_point_grid_real = sel_fun_point_grid[:,loc_z].T
sel_fun_z_real = sel_fun_z[loc_z]

im_sel = plt.imshow(sel_fun_point_grid_real.T, extent=[sel_fun_z_real.min(), sel_fun_z_real.max(), sel_fun_imag.max(), sel_fun_imag.min()], aspect='auto', vmin=0)
plt.colorbar(im_sel)

In [ ]:
sel_fun_point_grid_real.shape

##### QLF from Richards et al 2006

In [ ]:
from astropy.cosmology import LambdaCDM as LCDM
cosmo = LCDM(H0=70, Om0=0.3, Ode0=0.7)  ## they could have calculated luminosity distance differently
d_z2 = cosmo.luminosity_distance(z=2)

def QLF_phi_app(m, z, **kwargs):
    M_z2 = m - 5 * np.log10(cosmo.luminosity_distance(z=4)/(1E-5 * u.Mpc))
    phi_ast = pow(10, kwargs['log_phi_ast'])
    zeta = np.log10((1+z)/(1+kwargs['z_ref']))
    mu = M_z2 - (kwargs['M_ast'] + kwargs['B1']*zeta + kwargs['B2']*pow(zeta,2) + kwargs['B3']*pow(zeta,3))
    phi = phi_ast * pow(10, mu* (kwargs['A1'] + kwargs['A2'] * (z - 2.45)))

    return phi

def QLF_phi_abs(M, z, **kwargs):
    phi_ast = pow(10, kwargs['log_phi_ast'])
    zeta = np.log10((1+z)/(1+kwargs['z_ref']))
    mu = M - (kwargs['M_ast'] + kwargs['B1']*zeta + kwargs['B2']*pow(zeta,2) + kwargs['B3']*pow(zeta,3))
    phi = phi_ast * pow(10, mu* (kwargs['A1'] + kwargs['A2'] * (z - 2.45)))

    return phi   


In [ ]:
keywords = {'A1':0.83, 'A2':-0.11, 'B1':1.43, 'B2':36.63, 'B3':34.39, 'M_ast':-26, 'z_ref':2.45, 'log_phi_ast':-5.70}

z_array = np.linspace(0, 5, 100)
phi_array = QLF_phi_app(15, z_array, **keywords)

plt.plot(z_array, phi_array)
plt.yscale('log')

In [ ]:
m_array = sel_fun_imag
z_array = sel_fun_z[(sel_fun_z >= 2.9) & (sel_fun_z <= 5.4)]
m_grid, z_grid = np.meshgrid(m_array, z_array)
phi_array1 = QLF_phi_app(m_grid, z_grid, **keywords)

im = plt.imshow(phi_array1, extent=[z_array.min(), z_array.max(), m_array.min(), m_array.max()], norm='log', aspect='auto')
plt.colorbar(im)

In [ ]:
M_array = np.linspace(-15.0, -20.2, 100)
z_array = np.linspace(2.9, 5.4, 1000)

M_grid, z_grid = np.meshgrid(m_array, z_array)
phi_array2 = QLF_phi_abs(M_grid, z_grid, **keywords)

im1 = plt.imshow(phi_array2, extent=[z_array.min(), z_array.max(), M_array.min(), M_array.max()], norm='log', aspect='auto')
plt.colorbar(im1)

### Making a new random catalog via the QLF with each magnitude bin considered seperately

In [ ]:
from astropy.cosmology import LambdaCDM as LCDM
cosmo = LCDM(H0=70, Om0=0.3, Ode0=0.7)  ## they could have calculated luminosity distance differently
d_z2 = cosmo.luminosity_distance(z=2)

def QLF_phi_app(m, z, **kwargs):
    M_z2 = m - 5 * np.log10(cosmo.luminosity_distance(z=4)/(1E-5 * u.Mpc))
    phi_ast = pow(10, kwargs['log_phi_ast'])
    zeta = np.log10((1+z)/(1+kwargs['z_ref']))
    mu = M_z2 - (kwargs['M_ast'] + kwargs['B1']*zeta + kwargs['B2']*pow(zeta,2) + kwargs['B3']*pow(zeta,3))
    phi = phi_ast * pow(10, mu* (kwargs['A1'] + kwargs['A2'] * (z - 2.45)))

    return phi.value

keywords = {'A1':0.83, 'A2':-0.11, 'B1':1.43, 'B2':36.63, 'B3':34.39, 'M_ast':-26, 'z_ref':2.45, 'log_phi_ast':-5.70}

sdss_sel_fun_file_name = DP2_DIRECTORY + 'Data/sdss_q_sel_fun_richards.dat'
sdss_sel_fun_file = open(sdss_sel_fun_file_name, 'r')
sdss_sel_fun = np.genfromtxt(sdss_sel_fun_file)

sel_fun_imag_col = sdss_sel_fun[:,0]
sel_fun_z_col = sdss_sel_fun[:,1]
sel_fun_point_col = sdss_sel_fun[:,2]
sel_fun_radio_col = sdss_sel_fun[:,3]
sel_fun_extended_col = sdss_sel_fun[:,4]

### selection function is given in the following way: 53 i magnitudes are there from 15.0 to 20.2, difference of 0.1, for each magnitude 121 z values 
# from 0.0 to 6.0 is given with difference of 0.05.

sel_fun_z = sel_fun_z_col[0:121] ### this is the z values where we have the S(z)
sel_fun_imag = np.reshape(sel_fun_imag_col, (53,121))[:,0]
sel_fun_point_val = np.sum(np.reshape(sel_fun_point_col,(53, 121)), axis=0) * 0.1
sel_fun_point_grid = np.reshape(sel_fun_point_col,(53, 121))
sel_fun_radio_val = np.sum(np.reshape(sel_fun_radio_col,(53, 121)), axis=0) * 0.1
sel_fun_ext_val = np.sum(np.reshape(sel_fun_extended_col,(53, 121)), axis=0) * 0.1

loc_29_54 = np.where((sel_fun_z >= 2.9) & (sel_fun_z <= 5.05))
sel_fun_z_act= sel_fun_z[loc_29_54][1:-1]
sel_fun_z_bins = (sel_fun_z[loc_29_54] - 0.05/2)[1:]
sel_fun_point_act = (sel_fun_point_val[loc_29_54])[1:-1]

def make_rand_from_pz(pz, bins, n_samples):
    hist, edges = pz, bins
    bin_widths = np.diff(edges)
    cdf = np.cumsum(hist * bin_widths) / np.sum(hist * bin_widths)
    #plt.stairs(cdf, bins)

    # Generate new z values that follow the histogram distribution
    uniform_values = np.random.rand(n_samples)
    bin_indices = np.searchsorted(cdf, uniform_values)
    bin_edges = edges[bin_indices-1]
    bin_diff = edges[bin_indices] - edges[bin_indices-1]
    bin_weights = (uniform_values - cdf[bin_indices-1]) / (cdf[bin_indices] - cdf[bin_indices-1])

    new_z_values = bin_edges + bin_weights * bin_diff

    return new_z_values

def make_rand_from_n_m_z(n_mz, z_bins, n_samples):
    n_mz[np.where(n_mz < 0)] = 0
    n_mz = n_mz/(np.sum(np.sum(n_mz, axis=0) * 0.1, axis=0) * 0.05)
    z_samples = np.array([])
    for i in range(len(n_mz)-1):
        temp_samp = int(np.sum(n_mz[i]) * 0.05 * 0.1 * n_samples + 1)
        temp_pz = n_mz[i]/(np.sum(n_mz[i]) * 0.05)
        z_samples = np.concatenate((z_samples, make_rand_from_pz(temp_pz, z_bins, temp_samp)))

    np.random.shuffle(z_samples)
    return z_samples[:n_samples]

In [ ]:
m_values, z_values = np.meshgrid(sel_fun_imag, sel_fun_z)
m_values

In [ ]:
phi_array = QLF_phi_app(m_values, z_values, **keywords)
phi_array = np.sum(phi_array, axis=1) * (sel_fun_imag[1] - sel_fun_imag[0])
plt.plot(sel_fun_z, phi_array)
plt.yscale('log')
plt.ylim((1E-9, 1E-7))
plt.xlim((2.9, 5.00))

In [ ]:
loc = np.where((sel_fun_z_col >= 2.95) & (sel_fun_z_col <= 5.35))
sel_fun_array = sel_fun_point_col[loc]
sel_fun_array = sel_fun_array.reshape(53, 49)

zarr, magarr = np.meshgrid(sel_fun_z_act, sel_fun_imag)

qlf_array = QLF_phi_app(magarr, zarr, **keywords)

n_arr = sel_fun_array * qlf_array

In [ ]:
num = np.sum(n_arr, axis=0) * 0.1
denom = np.sum(num) * 0.05
n_z = num/denom
plt.plot(sel_fun_z_act, n_z)

rand_ra_col_shen, rand_dec_col_shen, rand_red_col_shen = make_shen_random_catalog_v3(ra_col_shen, dec_col_shen, red_col_shen, 5, 10)
plt.close()
rand_red_col_shen = make_rand_from_pz(n_z, sel_fun_z_bins, len(rand_ra_col_shen))

try_clustering_for_z_dist(rand_ra_col_shen, rand_dec_col_shen, rand_red_col_shen)

In [ ]:
loc = np.where((sel_fun_z_col >= 2.95) & (sel_fun_z_col <= 5.00))
sel_fun_z_act = sel_fun_z[np.where((sel_fun_z >= 2.95) & (sel_fun_z <= 5.00))]
z_len = len(sel_fun_z_act)
sel_fun_array = sel_fun_point_col[loc]
sel_fun_array = sel_fun_array.reshape(53, z_len)

loc_29_54 = np.where((sel_fun_z >= 2.9) & (sel_fun_z <= 5.05))
sel_fun_z_bins = (sel_fun_z[loc_29_54] - 0.05/2)[1:]

zarr, magarr = np.meshgrid(sel_fun_z_act, sel_fun_imag)

qlf_array = QLF_phi_app(magarr, zarr, **keywords)

n_arr = sel_fun_array * qlf_array

rand_ra_col_shen, rand_dec_col_shen, rand_red_col_shen = make_shen_random_catalog_v2(ra_col_shen, dec_col_shen, red_col_shen, 5, 10)
plt.close()
rand_red_col_shen = make_rand_from_n_m_z(n_arr, sel_fun_z_bins, len(rand_ra_col_shen))

try_clustering_for_z_dist(rand_ra_col_shen, rand_dec_col_shen, rand_red_col_shen)

### Trying a new random catalog with z distribution exactly same as data catalog

In [ ]:
rand_ra_col_shen, rand_dec_col_shen, rand_red_col_shen = make_shen_random_catalog_v2(ra_col_shen, dec_col_shen, red_col_shen, 5, 10)
plt.close()
rand_red_col_shen = np.tile(red_col_shen, (10,1))
rand_red_col_shen = rand_red_col_shen.flatten()
np.random.shuffle(rand_red_col_shen)
try_clustering_for_z_dist(rand_ra_col_shen, rand_dec_col_shen, rand_red_col_shen)

## Analysing the result that we got